In [43]:
random_seed=7
from numpy.random import seed
seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)

In [44]:
import tensorflow as tp
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import pandas as pd
from keras import backend as K
# from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

In [45]:
from sklearn.model_selection import StratifiedKFold

In [46]:
#checking if GPU is used 
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [47]:
dataset = pd.read_csv("../../data/1.csv", delimiter=",")
#dataset

In [48]:
def mean_imputation(dataset): 
    """"""
    imputed_dataset=dataset.fillna(dataset.mean())
    return imputed_dataset

In [49]:
dataset=mean_imputation(dataset)
#dataset

In [50]:
#to _numpy() works for 0.24.2 + version of pandas
pd.__version__

'0.24.2'

In [51]:
#dataset_np_array=dataset.to_numpy()
dataset_np_array=dataset.values
dataset_np_array

array([[ 18. ,   8. , 307. , ...,  12. ,  70. ,   1. ],
       [ 15. ,   8. , 350. , ...,  11.5,  70. ,   1. ],
       [ 18. ,   8. , 318. , ...,  11. ,  70. ,   1. ],
       ...,
       [ 32. ,   4. , 135. , ...,  11.6,  82. ,   1. ],
       [ 28. ,   4. , 120. , ...,  18.6,  82. ,   1. ],
       [ 31. ,   4. , 119. , ...,  19.4,  82. ,   1. ]])

In [52]:
#split into input (X) and output (Y)
def splitXY(dataset): 
    """
    Takes numpy array as input and converts first column into Y and rest into X
    """
    Y=dataset[:,0]
    X=dataset[:,1:]
    return X,Y

X,Y= splitXY(dataset_np_array)

In [53]:
m,n=X.shape
X1=X[:,0:7]
X1

array([[  8. , 307. , 130. , ...,  12. ,  70. ,   1. ],
       [  8. , 350. , 165. , ...,  11.5,  70. ,   1. ],
       [  8. , 318. , 150. , ...,  11. ,  70. ,   1. ],
       ...,
       [  4. , 135. ,  84. , ...,  11.6,  82. ,   1. ],
       [  4. , 120. ,  79. , ...,  18.6,  82. ,   1. ],
       [  4. , 119. ,  82. , ...,  19.4,  82. ,   1. ]])

In [54]:
def forward_sel(X,Y):
    m,n=X.shape
    for colno in range(0,n):
        X=X[:,0:colno+1]
        #run model from here..
    return None 

In [114]:
#showing error... this method is for classification..
def cross_validation():
    ##try cross validate for regressor, but without one..
    # define 10-fold cross validation test harness
    kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
    cvscores = []
    for train, test in kfold.split(X, Y):
        model=fit_model(X[train],Y[train])
        # evaluate the model
        dict_scores=eval_model(X,Y,model)
        cvscores.append(dict_scores)

    for dict_scores in cvscores:
        for k,v in dict_scores:
            print("%.2f%% (+/- %.2f%%)" % (k, v))
#cross_validation()

In [55]:
def r_squared(y_true, y_pred):
    
    sse=tp.reduce_sum(tp.square(tp.subtract(y_pred,y_true)))
    #using sst as y_true-y_mean 's squared mean
    y_mean= tp.reduce_mean(y_true)
    sst=tp.reduce_sum(tp.square(tp.subtract(y_true,y_mean)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    
    """
    m=tp.to_float(tp.size(y_true))
    y_true_sum_sq=(tp.square(tp.reduce_sum(y_true)))
    y_sq_mean=tp.divide(y_true_sum_sq,m)
    sst=tp.subtract(tp.tensordot(y_true,y_true,0),m*tp.square(tp.reduce_mean(y_true)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    """
    return r_square

In [101]:
def adj_r_squared(y_true, y_pred):
    sse=tp.reduce_sum(tp.square(tp.subtract(y_pred,y_true)))
    #using sst as y_true-y_mean 's squared mean
    y_mean= tp.reduce_mean(y_true)
    sst=tp.reduce_sum(tp.square(tp.subtract(y_true,y_mean)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    
    #n=model.count_params()#this part was working before but now is not..how to get model here though?
    #n=sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])
    n=tp.to_float(count)
    #m=tp.to_float(tp.size(y_true))
    rdf=tp.divide(tp.subtract(tp.to_float(instances),float(1)),tp.subtract(tp.to_float(instances),n))
    r_adj_square=tp.subtract(float(1),tp.tensordot(rdf,tp.subtract(float(1),r_square),0))
    return r_adj_square

In [102]:
def perceptronModel(X_dim=7,activation_name='sigmoid'):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(1, input_dim=X_dim,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    global count
    count=model.count_params()
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    sgd=optimizers.SGD(lr=0.05, momentum=0.02, decay=0.0, nesterov=False)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [103]:
def neuralNetwork3LModel(X_dim,activation_name='relu'):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(X_dim, input_dim=X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    global count
    count=model.count_params()#-X_dim+1
    sgd=optimizers.SGD(lr=0.05, momentum=0.02, decay=0.0, nesterov=False)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [108]:
def neuralNetwork5LModel(X_dim,activation_name='sigmoid'):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(X_dim, input_dim=X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    global count
    count=model.count_params() #-3*X_dim +1
    print('Count of parameters :'+str(count))
    sgd=optimizers.SGD(lr=0.1, momentum=0.02, decay=0.0, nesterov=False)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [109]:
def fit_model(X,Y):
    
    (X_size,X_dim)=X.shape
    global instances
    instances=X_size
    #model =perceptronModel(X_dim)
    #model =neuralNetwork3LModel(X_dim)
    model =neuralNetwork5LModel(X_dim)
    count=model.count_params() #-3*X_dim +1
    print('Count of parameters :'+str(count))
    model.summary()

    # Fit the model
    model.fit(X, Y, epochs=1000, batch_size=30)

    # evaluate the model
    # evaluate model with standardized dataset
    #estimator = KerasRegressor(build_fn=perceptronModel, epochs=100, batch_size=5, verbose=0)

    # kfold = KFold(n_splits=10, random_state=seed)
    # results = cross_val_score(estimator, X, Y, cv=kfold)
    # print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
    return model
model=fit_model(X,Y)

Count of parameters :176
Count of parameters :176
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 7)                 56        
_________________________________________________________________
dense_37 (Dense)             (None, 7)                 56        
_________________________________________________________________
dense_38 (Dense)             (None, 7)                 56        
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 8         
Total params: 176
Trainable params: 176
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
406/406 [==============================] - 1s 2ms/step - loss: 581.3899 - mean_squared_error: 581.3899 - r_squared: -9.2694 - adj_r_squared: -17.0831
Epoch 2/1000
406/406 [==============================] - 0s 141u

406/406 [==============================] - 0s 123us/step - loss: 309.5204 - mean_squared_error: 309.5204 - r_squared: -4.7846 - adj_r_squared: -9.1859
Epoch 46/1000
406/406 [==============================] - 0s 116us/step - loss: 304.4884 - mean_squared_error: 304.4884 - r_squared: -4.4965 - adj_r_squared: -8.6786
Epoch 47/1000
406/406 [==============================] - 0s 120us/step - loss: 299.6428 - mean_squared_error: 299.6428 - r_squared: -4.3111 - adj_r_squared: -8.3522
Epoch 48/1000
406/406 [==============================] - 0s 115us/step - loss: 294.9519 - mean_squared_error: 294.9519 - r_squared: -4.3777 - adj_r_squared: -8.4694
Epoch 49/1000
406/406 [==============================] - 0s 120us/step - loss: 290.5073 - mean_squared_error: 290.5073 - r_squared: -4.1212 - adj_r_squared: -8.0177
Epoch 50/1000
406/406 [==============================] - 0s 108us/step - loss: 286.2074 - mean_squared_error: 286.2074 - r_squared: -4.1902 - adj_r_squared: -8.1392
Epoch 51/1000
406/406 [=

406/406 [==============================] - 0s 115us/step - loss: 170.8361 - mean_squared_error: 170.8361 - r_squared: -2.0299 - adj_r_squared: -4.3352
Epoch 95/1000
406/406 [==============================] - 0s 113us/step - loss: 169.1218 - mean_squared_error: 169.1218 - r_squared: -2.0640 - adj_r_squared: -4.3954
Epoch 96/1000
406/406 [==============================] - 0s 111us/step - loss: 167.4288 - mean_squared_error: 167.4288 - r_squared: -1.9851 - adj_r_squared: -4.2563
Epoch 97/1000
406/406 [==============================] - 0s 115us/step - loss: 165.7769 - mean_squared_error: 165.7769 - r_squared: -1.9053 - adj_r_squared: -4.1158
Epoch 98/1000
406/406 [==============================] - 0s 113us/step - loss: 164.1003 - mean_squared_error: 164.1003 - r_squared: -1.9068 - adj_r_squared: -4.1186
Epoch 99/1000
406/406 [==============================] - 0s 106us/step - loss: 162.4754 - mean_squared_error: 162.4754 - r_squared: -2.0536 - adj_r_squared: -4.3771
Epoch 100/1000
406/406 [

406/406 [==============================] - 0s 118us/step - loss: 108.5494 - mean_squared_error: 108.5494 - r_squared: -0.8875 - adj_r_squared: -2.3236
Epoch 144/1000
406/406 [==============================] - 0s 118us/step - loss: 107.6594 - mean_squared_error: 107.6594 - r_squared: -0.8705 - adj_r_squared: -2.2937
Epoch 145/1000
406/406 [==============================] - 0s 115us/step - loss: 106.7710 - mean_squared_error: 106.7710 - r_squared: -0.8551 - adj_r_squared: -2.2665
Epoch 146/1000
406/406 [==============================] - 0s 113us/step - loss: 105.9050 - mean_squared_error: 105.9050 - r_squared: -0.8341 - adj_r_squared: -2.2296
Epoch 147/1000
406/406 [==============================] - 0s 111us/step - loss: 105.0531 - mean_squared_error: 105.0531 - r_squared: -0.8551 - adj_r_squared: -2.2666
Epoch 148/1000
406/406 [==============================] - 0s 116us/step - loss: 104.2033 - mean_squared_error: 104.2033 - r_squared: -0.7928 - adj_r_squared: -2.1569
Epoch 149/1000
406/

406/406 [==============================] - 0s 118us/step - loss: 77.3483 - mean_squared_error: 77.3483 - r_squared: -0.3763 - adj_r_squared: -1.4234
Epoch 193/1000
406/406 [==============================] - 0s 106us/step - loss: 76.9291 - mean_squared_error: 76.9291 - r_squared: -0.3114 - adj_r_squared: -1.3093
Epoch 194/1000
406/406 [==============================] - 0s 115us/step - loss: 76.5154 - mean_squared_error: 76.5154 - r_squared: -0.3267 - adj_r_squared: -1.3362
Epoch 195/1000
406/406 [==============================] - 0s 113us/step - loss: 76.1286 - mean_squared_error: 76.1286 - r_squared: -0.3232 - adj_r_squared: -1.3300
Epoch 196/1000
406/406 [==============================] - 0s 111us/step - loss: 75.7356 - mean_squared_error: 75.7356 - r_squared: -0.2970 - adj_r_squared: -1.2839
Epoch 197/1000
406/406 [==============================] - 0s 106us/step - loss: 75.3679 - mean_squared_error: 75.3679 - r_squared: -0.2876 - adj_r_squared: -1.2673
Epoch 198/1000
406/406 [=======

...

406/406 [==============================] - 0s 77us/step - loss: 27.9706 - mean_squared_error: 27.9706 - r_squared: 0.5171 - adj_r_squared: 0.1496
Epoch 790/1000
406/406 [==============================] - 0s 115us/step - loss: 27.9318 - mean_squared_error: 27.9318 - r_squared: 0.4947 - adj_r_squared: 0.1102
Epoch 791/1000
406/406 [==============================] - 0s 109us/step - loss: 28.1431 - mean_squared_error: 28.1431 - r_squared: 0.4926 - adj_r_squared: 0.1066
Epoch 792/1000
406/406 [==============================] - 0s 77us/step - loss: 27.6758 - mean_squared_error: 27.6758 - r_squared: 0.5195 - adj_r_squared: 0.1540
Epoch 793/1000
406/406 [==============================] - 0s 77us/step - loss: 28.0671 - mean_squared_error: 28.0671 - r_squared: 0.4981 - adj_r_squared: 0.1162
Epoch 794/1000
406/406 [==============================] - 0s 115us/step - loss: 27.6875 - mean_squared_error: 27.6875 - r_squared: 0.5199 - adj_r_squared: 0.1547
Epoch 795/1000
406/406 [======================

406/406 [==============================] - 0s 108us/step - loss: 27.9272 - mean_squared_error: 27.9272 - r_squared: 0.5209 - adj_r_squared: 0.1564
Epoch 840/1000
406/406 [==============================] - 0s 106us/step - loss: 28.2477 - mean_squared_error: 28.2477 - r_squared: 0.5169 - adj_r_squared: 0.1494
Epoch 841/1000
406/406 [==============================] - 0s 103us/step - loss: 28.2189 - mean_squared_error: 28.2189 - r_squared: 0.5154 - adj_r_squared: 0.1467
Epoch 842/1000
406/406 [==============================] - 0s 165us/step - loss: 28.1148 - mean_squared_error: 28.1148 - r_squared: 0.5068 - adj_r_squared: 0.1315
Epoch 843/1000
406/406 [==============================] - 0s 115us/step - loss: 28.1050 - mean_squared_error: 28.1050 - r_squared: 0.5029 - adj_r_squared: 0.1247
Epoch 844/1000
406/406 [==============================] - 0s 103us/step - loss: 27.9431 - mean_squared_error: 27.9431 - r_squared: 0.5035 - adj_r_squared: 0.1257
Epoch 845/1000
406/406 [===================

406/406 [==============================] - 0s 101us/step - loss: 27.5286 - mean_squared_error: 27.5286 - r_squared: 0.5259 - adj_r_squared: 0.1651
Epoch 890/1000
406/406 [==============================] - 0s 98us/step - loss: 27.5175 - mean_squared_error: 27.5175 - r_squared: 0.5166 - adj_r_squared: 0.1488
Epoch 891/1000
406/406 [==============================] - 0s 98us/step - loss: 27.5089 - mean_squared_error: 27.5089 - r_squared: 0.5223 - adj_r_squared: 0.1589
Epoch 892/1000
406/406 [==============================] - 0s 98us/step - loss: 27.5098 - mean_squared_error: 27.5098 - r_squared: 0.5152 - adj_r_squared: 0.1464
Epoch 893/1000
406/406 [==============================] - 0s 101us/step - loss: 27.4699 - mean_squared_error: 27.4699 - r_squared: 0.5091 - adj_r_squared: 0.1356
Epoch 894/1000
406/406 [==============================] - 0s 98us/step - loss: 27.4522 - mean_squared_error: 27.4522 - r_squared: 0.5065 - adj_r_squared: 0.1310
Epoch 895/1000
406/406 [=======================

406/406 [==============================] - 0s 106us/step - loss: 27.0473 - mean_squared_error: 27.0473 - r_squared: 0.5167 - adj_r_squared: 0.1490
Epoch 940/1000
406/406 [==============================] - 0s 108us/step - loss: 27.9781 - mean_squared_error: 27.9781 - r_squared: 0.5186 - adj_r_squared: 0.1524
Epoch 941/1000
406/406 [==============================] - 0s 103us/step - loss: 26.9166 - mean_squared_error: 26.9166 - r_squared: 0.5185 - adj_r_squared: 0.1521
Epoch 942/1000
406/406 [==============================] - 0s 108us/step - loss: 26.8728 - mean_squared_error: 26.8728 - r_squared: 0.5378 - adj_r_squared: 0.1861
Epoch 943/1000
406/406 [==============================] - 0s 115us/step - loss: 26.9768 - mean_squared_error: 26.9768 - r_squared: 0.5131 - adj_r_squared: 0.1427
Epoch 944/1000
406/406 [==============================] - 0s 130us/step - loss: 26.9964 - mean_squared_error: 26.9964 - r_squared: 0.5189 - adj_r_squared: 0.1528
Epoch 945/1000
406/406 [===================

406/406 [==============================] - 0s 123us/step - loss: 26.9208 - mean_squared_error: 26.9208 - r_squared: 0.5405 - adj_r_squared: 0.1908
Epoch 990/1000
406/406 [==============================] - 0s 115us/step - loss: 27.2656 - mean_squared_error: 27.2656 - r_squared: 0.5222 - adj_r_squared: 0.1586
Epoch 991/1000
406/406 [==============================] - 0s 110us/step - loss: 27.0924 - mean_squared_error: 27.0924 - r_squared: 0.5176 - adj_r_squared: 0.1505
Epoch 992/1000
406/406 [==============================] - 0s 113us/step - loss: 26.8475 - mean_squared_error: 26.8475 - r_squared: 0.5218 - adj_r_squared: 0.1579
Epoch 993/1000
406/406 [==============================] - 0s 109us/step - loss: 26.9495 - mean_squared_error: 26.9495 - r_squared: 0.5166 - adj_r_squared: 0.1488
Epoch 994/1000
406/406 [==============================] - 0s 108us/step - loss: 27.0210 - mean_squared_error: 27.0210 - r_squared: 0.5291 - adj_r_squared: 0.1708
Epoch 995/1000
406/406 [===================

In [106]:
def eval_model(X,Y,model):
#model evaluation.

    scores = model.evaluate(X, Y, verbose=0)
    dict_scores={}
    for i in range(len(scores)):
        dict_scores[model.metrics_names[i+1]]=score[i+1]
        
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    return dict_scores
eval_model(X,Y,model)

mean_squared_error: 19.13%
r_squared: 40.62%
adj_r_squared: 29.68%


In [70]:
#the model is not learning
#what can be wrong? 
# basic mistake can be how X and Y are designed, but this is unlikely.
#somehow 